In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("./Batch_5064543_batch_results_50.csv")

In [8]:
print(df.columns)
'''
The needed columns for querying: 
    1. HITTId-> The unique ID for the sentence 
    2. Input.utterance -> Sentence
    3. Input.Instance  -> Target
    4. ِ[Answer.object1, Answer.object8] -> Objects
    5. [Answer.order1, Answer.order8] -> Order
'''

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.id',
       'Input.stimulus_id', 'Input.utterance', 'Input.correct_guess',
       'Input.speaker_id', 'Input.listener_id', 'Input.scan_id',
       'Input.instance_type', 'Input.target_id', 'Input.tokens',
       'Input.dataset', 'Input.mentions_target_class',
       'Input.uses_object_lang', 'Input.uses_spatial_lang',
       'Input.uses_color_lang', 'Input.uses_shape_lang', 'Answer.object1',
       'Answer.object2', 'Answer.object3', 'Answer.o

'\nThe needed columns for querying: \n    1. HITTId-> The unique ID for the sentence \n    2. Input.utterance -> Sentence\n    3. Input.Instance  -> Target\n    4. ِ[Answer.object1, Answer.object8] -> Objects\n    5. [Answer.order1, Answer.order8] -> Order\n'

# With Taking into account the order

In [56]:
#TODO 1: Check whether all Annotation have the same order or not 

#Get Unique HITTId
unique_HITTId = df['HITId'].unique()
total_number_of_examples = df.shape[0]

df_copy = df.copy()

order_is_wrong_by_all_HITTId = []
df_copy = df_copy.sort_values(by=['HITId'])

#Mapper to know the number of HITTId for each sentence 
sentence_Num_HTTHId_mapper = {}

for i in range(total_number_of_examples):
    if df_copy['HITId'].iloc[i] not in sentence_Num_HTTHId_mapper:
        sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[i]] = 1
    else:
        sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[i]] += 1
        
#Check whether all Annotation have the same order or not for each HITTId

Number_of_agreements_per_HITTId_for_both_flexible= {}  
Number_of_agreements_per_HITTId_for_both_strict = {}
idx = 0
while(idx < 1):

    both_list    = set()
    both_list_strict = set()

    both_count = {}
    both_count_strict = {}
    i = idx 
    stop_at = i + sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[i]]
    print(i, stop_at)
    while(i < stop_at):
        
        Answers_generically  = []
        Answers_generically_strict = []
        for j in range(1,9):
            feature1 = 'Answer.order' + str(j)
            feature2 = 'Answer.object' + str(j)
                              
            if str(df_copy[feature2].iloc[i]) != 'nan' and str(df_copy[feature1].iloc[i]) != 'nan':
                object_ = str(df_copy[feature2].iloc[i])
                object_ = object_.lower()
                import string
                object_ = object_.translate(str.maketrans('', '', string.punctuation))
                #TODO: Make it more flexiable
                object_ = object_.split(" ")[-1]
                try:
                    Answers_generically.append((int(df_copy[feature1].iloc[i]), str(object_)))
                    Answers_generically_strict.append((int(df_copy[feature1].iloc[i]), str(object_)))
                except: 
                    print("I WILL NOT ADD THIS BECAUSE OF THE ERROR: ", df_copy['HITId'].iloc[i])

        
        #Order Answers_for_each_annotator_by_order
        # Answers_for_each_annotator_by_order.sort()
        # Answers_for_each_annotator_by_object.sort()
        # print(Answers_generically)
        # Answers_generically_strict = Answers_generically.copy()
        Answers_generically.sort() #Sort by order then object
        
        if (str(Answers_generically_strict) not in both_count_strict):
            both_count_strict[str(Answers_generically_strict)] = 1
        else:
            both_count_strict[str(Answers_generically_strict)] += 1
            
            
        
        if(str(Answers_generically) not in both_count):
            both_count[str(Answers_generically)] = 1
        else:
            both_count[str(Answers_generically)] += 1
            
        both_list.add(str(Answers_generically))
        both_list_strict.add(str(Answers_generically_strict))
        i+=1
        
    #Get the maximum Voting for each using otdered_list and ordere_list_count
    max_for_both = 0
    max_for_both_strict = 0
    
    print(both_count_strict, ' ' , both_count)
    for both in both_count_strict:
        max_for_both_strict = max(max_for_both_strict, both_count_strict[both])
    
    for both in both_count:
        max_for_both = max(max_for_both, both_count[both])
        
    
    #Add it to the agreement 
    if sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[idx]] == 4:
        Number_of_agreements_per_HITTId_for_both_flexible[df_copy['HITId'].iloc[idx]] = max_for_both
        Number_of_agreements_per_HITTId_for_both_strict[df_copy['HITId'].iloc[idx]] = max_for_both_strict
    
    
    idx += sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[idx]]

0 4
{"[(2, 'door'), (1, 'room')]": 1, "[(1, 'door'), (2, 'room')]": 1, "[(2, 'door'), (1, 'doors')]": 1, "[(1, 'doors'), (2, 'door')]": 1}   {"[(1, 'room'), (2, 'door')]": 1, "[(1, 'door'), (2, 'room')]": 1, "[(1, 'doors'), (2, 'door')]": 2}


#Does not Follow the order!

In [13]:
#TODO 2: 
    #how many samples are completed fully (by 4 annotators) and how many samples are completed by only 3 annotators?

#Using sentence_Num_HTTHId_mapper

sentences_done_with_4_annotators = 0
sentences_done_with_3_annotators = 0

for sentence in sentence_Num_HTTHId_mapper:
    if sentence_Num_HTTHId_mapper[sentence] == 4:
        sentences_done_with_4_annotators += 1
    elif sentence_Num_HTTHId_mapper[sentence] == 3:
        sentences_done_with_3_annotators += 1
        

print("sentences_done_with_4_annotators: ", sentences_done_with_4_annotators)
print("sentences_done_with_3_annotators: ", sentences_done_with_3_annotators)

sentences_done_with_4_annotators:  1650
sentences_done_with_3_annotators:  160


In [14]:
len(sentence_Num_HTTHId_mapper)

2692

In [15]:

#TODO 3: The Percange for each case: Flexible 
percentage_4_0_agreement = 0 
precentage_3_1_agreement = 0

for sentence in Number_of_agreements_per_HITTId_for_both_flexible:
    if Number_of_agreements_per_HITTId_for_both_flexible[sentence] == 4:
        percentage_4_0_agreement += 1
    elif Number_of_agreements_per_HITTId_for_both_flexible[sentence] == 3:
        precentage_3_1_agreement += 1

print("percentage_4_0_agreement: ", percentage_4_0_agreement/sentences_done_with_4_annotators)
print("precentage_3_1_agreement: ", precentage_3_1_agreement/sentences_done_with_4_annotators)


percentage_4_0_agreement:  0.4593939393939394
precentage_3_1_agreement:  0.25333333333333335


In [16]:
0.4593939393939394 + 0.25333333333333335

0.7127272727272728

In [17]:
print("Number of Agreements with 4 annotators: ", percentage_4_0_agreement)
print("Number of Agreements with 3 annotators: ", precentage_3_1_agreement)

Number of Agreements with 4 annotators:  758
Number of Agreements with 3 annotators:  418


In [18]:

#TODO 3: The Percange for each case: Strict 
percentage_4_0_agreement = 0 
precentage_3_1_agreement = 0

for sentence in  Number_of_agreements_per_HITTId_for_both_strict:
    if Number_of_agreements_per_HITTId_for_both_strict[sentence] == 4:
        percentage_4_0_agreement += 1
    elif Number_of_agreements_per_HITTId_for_both_strict[sentence] == 3:
        precentage_3_1_agreement += 1

print("percentage_4_0_agreement: ", percentage_4_0_agreement/sentences_done_with_4_annotators)
print("precentage_3_1_agreement: ", precentage_3_1_agreement/sentences_done_with_4_annotators)
# print(precentage_3_1_agreement)

percentage_4_0_agreement:  0.4309090909090909
precentage_3_1_agreement:  0.2381818181818182


In [19]:
print("Number of Agreements with 4 annotators: ", percentage_4_0_agreement)
print("Number of Agreements with 3 annotators: ", precentage_3_1_agreement)

Number of Agreements with 4 annotators:  711
Number of Agreements with 3 annotators:  393


# With Substring

In [35]:
df_copy = df.copy()
df_copy['HITId'].iloc[0]

'3QREJ3J433GK5HZ84AAP6VPESRLKLZ'

In [54]:
# #FOR SUBSTRING: WOTKING ON IT: 
# #TODO 1: Check whether all Annotation have the same order or not 

# #Get Unique HITTId
# unique_HITTId = df['HITId'].unique()
# total_number_of_examples = df.shape[0]

# df_copy = df.copy()

# order_is_wrong_by_all_HITTId = []
# df_copy = df_copy.sort_values(by=['HITId'])

# #Mapper to know the number of HITTId for each sentence 
# sentence_Num_HTTHId_mapper = {}

# for i in range(total_number_of_examples):
#     if df_copy['HITId'].iloc[i] not in sentence_Num_HTTHId_mapper:
#         sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[i]] = 1
#     else:
#         sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[i]] += 1
        
# #Check whether all Annotation have the same order or not for each HITTId

# Number_of_agreements_per_HITTId_for_both_flexible= {}  
# Number_of_agreements_per_HITTId_for_both_strict = {}
# idx = 0
# while(idx < 1):

#     both_list    = set()
#     both_list_strict = set()

#     both_count = {}
#     both_count_strict = {}
#     i = idx 
#     stop_at = i + sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[i]]
#     print(i, stop_at)
#     while(i < stop_at):
#         Answers_generically  = []
#         Answers_generically_strict = []
#         for j in range(1,9):
#             feature1 = 'Answer.order' + str(j)
#             feature2 = 'Answer.object' + str(j)
                              
#             if str(df_copy[feature2].iloc[i]) != 'nan' and str(df_copy[feature1].iloc[i]) != 'nan':
#                 object_ = str(df_copy[feature2].iloc[i])
#                 object_ = object_.lower()
                
#                 import string
#                 object_ = object_.translate(str.maketrans('', '', string.punctuation))
#                 #TODO: Make it more flexiable
#                 try:
#                     Answers_generically.append((int(df_copy[feature1].iloc[i]), str(object_), i))
#                     Answers_generically_strict.append((int(df_copy[feature1].iloc[i]), str(object_), i))
#                 except: 
#                     print("I WILL NOT ADD THIS BECAUSE OF THE ERROR: ", df_copy['HITId'].iloc[i])

        
#         Answers_generically.sort() #Sort by order then object
        
#         if (str(Answers_generically_strict) not in both_count_strict):
#             both_count_strict[str(Answers_generically_strict)] = 1
#         else:
#             both_count_strict[str(Answers_generically_strict)] += 1
            
        
#         if(str(Answers_generically) not in both_count):
#             both_count[str(Answers_generically)] = 1
#         else:
#             both_count[str(Answers_generically)] += 1
            
#         both_list.add(str(Answers_generically))
#         both_list_strict.add(str(Answers_generically_strict))
#         i+=1
        
#     #Get the maximum Voting for each using otdered_list and ordere_list_count
#     max_for_both = 0
#     max_for_both_strict = 0
#     which_idx_strict = -1
#     which_idx_flexible = -1
    
#     for both in both_count_strict:
#         if max_for_both_strict < both_count_strict[both]:
#             max_for_both_strict = both_count_strict[both]
#             which_idx_strict = both.split(",")[-1][:-2]
    
#     for both in both_count:
#         if max_for_both < both_count[both]:
#             max_for_both = both_count[both]
#             which_idx_flexible = both.split(",")[-1][:-2]
    
#     best_flexible = []
#     best_strict   = []
#     for key in list(both_count.keys()):
#         list_ = key.strip('][').split(', ')
#         #Strip the tuple
#         tuples = ()
#         res_list = []
#         list_idx = 0
#         for  x in list_:
#             tuple_ = x.strip('()').split(',')
#             tuples += (tuple_[0],)
#             list_idx+=1
#             if (list_idx%3 == 0):
#                 res_list.append(tuples)
#                 tuples = ()
#         best_flexible.append(res_list)
        
#     for key in list(both_count_strict.keys()):
#         list_ = key.strip('][').split(', ')
#         #Strip the tuple
#         tuples = ()
#         res_list = []
#         list_idx = 0
#         for  x in list_:
#             tuple_ = x.strip('()').split(',')
#             tuples += (tuple_[0],)
#             list_idx+=1
#             if (list_idx%3 == 0):
#                 res_list.append(tuples)
#                 tuples = ()
#         best_strict.append(res_list)
    
#     print(best_flexible)
#     print(best_strict)
    
#     #get objects from the best_flexible and best_strict
    
#     flexible_objects = []
#     for i in range(len(best_flexible[int(which_idx_flexible)])):
#         flexible_objects.append(best_flexible[int(which_idx_flexible)][i][1])
        
#     strict_objects   = []
#     for i in range(len(best_strict[int(which_idx_strict)])):
#         strict_objects.append(best_strict[int(which_idx_strict)][i][1])

#     print(flexible_objects, strict_objects)
    
#     #Check with the substrings
#     second_max_for_both = 0
#     second_max_for_both_strict = 0
#     for i, flexible_list in enumerate(best_flexible):
#         if(i == which_idx_flexible):
#                 continue 
#         objects = [flexible_list[i][1] for i in range(len(flexible_list))]
#         temp_max = 0
#         for obj1 in objects:
#             for best_obj in flexible_objects:
#                 if obj1 in best_obj or best_obj in obj1:
#                     temp_max += 1
        
#         if temp_max == len(objects):
#             second_max_for_both += 1
        
#     #Check with the substrings: Strict
    
#     for i, strict_list in enumerate(best_strict):
        
#         if(i == which_idx_strict):
#                 continue 
        
#         objects = strict_list[1]
#         temp_max = 0
#         for obj1 in objects:
#             for best_obj in strict_objects:
#                 if obj1 in best_obj or best_obj in obj1:
#                     temp_max += 1
        
#         if temp_max == len(objects):
#             second_max_for_both_strict += 1
    
#     print("second_max_for_both: ", second_max_for_both, "second_max_for_both_strict: ", second_max_for_both_strict)
    
#     #Add it to the agreement 
#     if sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[idx]] == 4:
#         Number_of_agreements_per_HITTId_for_both_flexible[df_copy['HITId'].iloc[idx]] = max_for_both
#         Number_of_agreements_per_HITTId_for_both_strict[df_copy['HITId'].iloc[idx]] = max_for_both_strict
    
    
#     idx += sentence_Num_HTTHId_mapper[df_copy['HITId'].iloc[idx]]

0 4
[[('1', "'room'", '0'), ('2', "'door'", '0')], [('1', "'door'", '1'), ('2', "'center of room'", '1')], [('1', "'doors'", '2'), ('2', "'door'", '2')], [('1', "'doors'", '3'), ('2', "'door'", '3')]]
[[('2', "'door'", '0'), ('1', "'room'", '0')], [('1', "'door'", '1'), ('2', "'center of room'", '1')], [('2', "'door'", '2'), ('1', "'doors'", '2')], [('1', "'doors'", '3'), ('2', "'door'", '3')]]
["'room'", "'door'"] ["'door'", "'room'"]
second_max_for_both:  1 second_max_for_both_strict:  0
